### Imports

In [24]:
from bs4 import BeautifulSoup
import requests
import json
import re
import string
from pymongo import MongoClient
from nltk.corpus import stopwords
import pyarabic.araby as araby
from pyarabic.number import ArNumbers
from tashaphyne.stemming import ArabicLightStemmer
import qalsadi.lemmatizer as lem
from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer

### Loading the BeautifulSoup object

In [25]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'}
result = requests.get('https://www.hespress.com/economie', headers=headers)
doc = BeautifulSoup(result.text, 'html.parser')

### Retrieving the divs that contain the links to articles

In [26]:
main_div = doc.find_all(name='div',attrs={'class':'posts-categoy'})
posts = main_div[0].find_all(name='div',attrs={'class':'card-img-top'})
articals = []
for article in posts:
    # links
    link = article.find(name='a').get('href')
    # article titles
    title = article.find(name='a').get('title')
    articals.append({title : link})

### Storing links and titles in a json file

In [27]:
with open('hess_article_links.json', 'w', encoding='utf-8') as f:
    data = []
    for article in articals:
        title = list(article.keys())[0]
        link = list(article.values())[0]
        data.append({'title': title, 'link': link})
    json.dump(data, f, ensure_ascii=False, indent=2)

### Retrieving the divs that contain the links to articles

In [28]:
with open('hess_article_links.json', 'r', encoding='utf-8') as f:
    article_content = []
    data = json.load(f)
    for article in data:
        article_result = requests.get(article['link'], headers=headers)
        article_doc = BeautifulSoup(article_result.text, 'html.parser')
        article_content.append(article_doc.find_all(name='div',attrs={'class':'article-content'})[0].find_all(name='p'))

### Storing articles and their respective titles in a json file

In [29]:
with open('hess_article_content.json', 'w', encoding='utf-8') as f:
    contents = []
    for i,content in enumerate(article_content):
        content = ''
        for p in article_content[i]:
            content += p.text
        contents.append({'title': list(data[i].values())[0], 'content': content})
    json.dump(contents, f, ensure_ascii=False, indent=2)

### Uploading the two json files in a MongoDB database

In [30]:
with open('hess_article_links.json', 'r', encoding='utf-8') as f1:
    links = json.load(f1)
with open('hess_article_content.json', 'r', encoding='utf-8') as f2:
    art_content = json.load(f2)
client = MongoClient('mongodb://localhost:27017/')
db = client['NLP_lab1']
coll1 = db['hess_article_links']
coll2 = db['hess_article_content']
coll1.drop()
coll1.insert_many(links)
coll2.drop()
coll2.insert_many(art_content)
client.close()

### Loading the first article for the NLP pipeline

In [31]:
text = art_content[0]['content']
text.encode('utf-8')
print("Raw text: ",text)

Raw text:  أعلنت المؤسسة العربية لضمان الاستثمار وائتمان الصادرات “ضمان” عن حلول المغرب في المركز السابع ضمن مؤشر “فيتش” المركب للمخاطر خلال 2023.وأشارت المؤسسة ذاتها، في بلاغ صادر عنها، إلى استقرار التصنيف السيادي لـ4 دول عربية، مع تحسن تصنيف الإمارات والسعودية وسلطنة عمان وقطر، مقابل تراجع تصنيف المغرب ومصر وتونس ولبنان، إضافة إلى تغير الرؤية المستقبلية لـ6 دول، وفق أهم 4 وكالات عالمية.وأفاد البلاغ بأن دول الخليج ثم المغرب فالأردن ومصر استحوذت على مقدمة الترتيب عربيا، في أغلب التقييمات المتعلقة بمؤشرات المخاطر بكل أنواعها؛ فيما أوضحت البيانات المرفقة به استقرار تصنيف المغرب سابعا في تصنيف الدول العربية في مؤشر “فيتش” المركب لمخاطر الدول بـ55.2 نقطة، أي فوق المتوسط (48.3 في المائة).وأكدت “ضمان” تراجع وضع الدول العربية في مؤشرات مخاطر الدولة المتعلقة بعمليات التصدير والاستثمارات المباشرة الصادرة عن وكالة كريديندو”، وفي مؤشر مخاطر التجارة الصادر عن وكالة “نيكسي” اليابانية؛ فيما شهد متوسط الترتيب العالمي لهذه الدول تحسنا في مؤشرات مخاطر الدولة الصادرة عن وكالات “أليانز ترايد” و”اتراديوس”

### Tokenization

In [32]:
tokens = araby.tokenize(text)
sent_tokens = araby.sentence_tokenize(text)
print("Word tokenized list: ", tokens)
print("Sentence tokenized list: ", sent_tokens)
print("Word tokenized list length: ", len(tokens))
print("Sentence tokenized list length: ", len(sent_tokens))

Word tokenized list:  ['أعلنت', 'المؤسسة', 'العربية', 'لضمان', 'الاستثمار', 'وائتمان', 'الصادرات', '“', 'ضمان', '”', 'عن', 'حلول', 'المغرب', 'في', 'المركز', 'السابع', 'ضمن', 'مؤشر', '“', 'فيتش', '”', 'المركب', 'للمخاطر', 'خلال', '2023', '.', 'وأشارت', 'المؤسسة', 'ذاتها', '،', 'في', 'بلاغ', 'صادر', 'عنها', '،', 'إلى', 'استقرار', 'التصنيف', 'السيادي', 'لـ4', 'دول', 'عربية', '،', 'مع', 'تحسن', 'تصنيف', 'الإمارات', 'والسعودية', 'وسلطنة', 'عمان', 'وقطر', '،', 'مقابل', 'تراجع', 'تصنيف', 'المغرب', 'ومصر', 'وتونس', 'ولبنان', '،', 'إضافة', 'إلى', 'تغير', 'الرؤية', 'المستقبلية', 'لـ6', 'دول', '،', 'وفق', 'أهم', '4', 'وكالات', 'عالمية', '.', 'وأفاد', 'البلاغ', 'بأن', 'دول', 'الخليج', 'ثم', 'المغرب', 'فالأردن', 'ومصر', 'استحوذت', 'على', 'مقدمة', 'الترتيب', 'عربيا', '،', 'في', 'أغلب', 'التقييمات', 'المتعلقة', 'بمؤشرات', 'المخاطر', 'بكل', 'أنواعها', '؛', 'فيما', 'أوضحت', 'البيانات', 'المرفقة', 'به', 'استقرار', 'تصنيف', 'المغرب', 'سابعا', 'في', 'تصنيف', 'الدول', 'العربية', 'في', 'مؤشر', '“', 'فيتش', 

### Removing punctuation

In [33]:
ar_punct = ''')(+`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”،.”…“–ـ”.'''
en_punct = string.punctuation
punct_lst = ar_punct + en_punct

In [34]:
tokens = [token for token in tokens if token not in punct_lst]
sent_tokens = [sent.translate(str.maketrans('', '', punct_lst)) for sent in sent_tokens]
print("Tokenized text without punctuation: ",tokens)
print("Sentence tokenized text without punctuation: ",sent_tokens)
print("Length of tokenized text without punctuation: ", len(tokens))
print("Length of sentence tokenized text without punctuation: ", len(sent_tokens))

Tokenized text without punctuation:  ['أعلنت', 'المؤسسة', 'العربية', 'لضمان', 'الاستثمار', 'وائتمان', 'الصادرات', 'ضمان', 'عن', 'حلول', 'المغرب', 'في', 'المركز', 'السابع', 'ضمن', 'مؤشر', 'فيتش', 'المركب', 'للمخاطر', 'خلال', '2023', 'وأشارت', 'المؤسسة', 'ذاتها', 'في', 'بلاغ', 'صادر', 'عنها', 'إلى', 'استقرار', 'التصنيف', 'السيادي', 'لـ4', 'دول', 'عربية', 'مع', 'تحسن', 'تصنيف', 'الإمارات', 'والسعودية', 'وسلطنة', 'عمان', 'وقطر', 'مقابل', 'تراجع', 'تصنيف', 'المغرب', 'ومصر', 'وتونس', 'ولبنان', 'إضافة', 'إلى', 'تغير', 'الرؤية', 'المستقبلية', 'لـ6', 'دول', 'وفق', 'أهم', '4', 'وكالات', 'عالمية', 'وأفاد', 'البلاغ', 'بأن', 'دول', 'الخليج', 'ثم', 'المغرب', 'فالأردن', 'ومصر', 'استحوذت', 'على', 'مقدمة', 'الترتيب', 'عربيا', 'في', 'أغلب', 'التقييمات', 'المتعلقة', 'بمؤشرات', 'المخاطر', 'بكل', 'أنواعها', 'فيما', 'أوضحت', 'البيانات', 'المرفقة', 'به', 'استقرار', 'تصنيف', 'المغرب', 'سابعا', 'في', 'تصنيف', 'الدول', 'العربية', 'في', 'مؤشر', 'فيتش', 'المركب', 'لمخاطر', 'الدول', 'بـ55', '2', 'نقطة', 'أي', 'فوق

### Removing stopwords

In [35]:
stop_words = set(stopwords.words('arabic'))
print("Stop words: ",stop_words)
tokens = [word for word in tokens if word not in stop_words]
sent_tokens = [''.join([x for x in re.split(r'(\W+)', sent) if x not in stop_words]) for sent in sent_tokens]

print("Tokenized text without stopwords: ",tokens)
print("Sentence tokenized text without stopwords: ",sent_tokens)
print("Length of tokenized text without stopwords: ", len(tokens))
print("Length of sentence tokenized text without stopwords: ", len(sent_tokens))

Stop words:  {'اللتين', 'ف', 'هَاتِه', 'شمال', 'كذلك', 'لا سيما', 'إنا', 'بهما', 'فلس', 'لبيك', 'ستون', 'وإن', 'ممن', 'آهٍ', 'مكانكم', 'إياهم', 'أول', 'عشرين', 'حجا', 'تِي', 'غداة', 'سبعمئة', 'لستم', 'رأى', 'هو', 'مايو', 'ذلك', 'انقلب', 'والذي', 'تشرين', 'شيكل', 'آهاً', 'حبيب', 'هيت', 'عدا', 'نا', 'ثمة', 'أربعاء', 'خلافا', 'عليك', 'لكم', 'منها', 'هن', 'حبذا', 'إلَيْكَ', 'تلكم', 'سوى', 'رُبَّ', 'اللواتي', 'لسن', 'تعلَّم', 'تفعلان', 'أنا', 'كليهما', 'كرب', 'هبّ', 'لم', 'أُفٍّ', 'فلا', 'حيَّ', 'بيد', 'أن', 'ك', 'إي', 'فيه', 'كان', 'فيها', 'ز', 'فوق', 'ثمّ', 'وا', 'أحد', 'ثلاثمئة', 'وهب', 'بخ', 'لن', 'ص', 'ذا', 'ءَ', 'هيا', 'هللة', 'إذن', 'ثلاثين', 'هاتان', 'أعطى', 'خامس', 'هذا', 'يا', 'خبَّر', 'آ', 'تعسا', 'جمعة', 'إيانا', 'بات', 'هاهنا', 'إليكَ', 'ذيت', 'إزاء', 'أمد', 'كما', 'ميم', 'بسّ', 'كسا', 'سوف', 'كأين', 'أطعم', 'أبٌ', 'دون', 'خميس', 'نيف', 'كاد', 'مئة', 'لو', 'نيسان', 'كى', 'أصلا', 'اربعين', 'أى', 'ؤ', 'سبعة', 'ته', 'سبعون', 'م', 'قاف', 'ج', 'أمّا', 'زعم', 'ألا', 'لات', 'ثان', 'سر

### Convert numbers to words

In [36]:
arnum = ArNumbers()
num_tokens = []
for i,token in enumerate(tokens):
    if tokens[i].isdigit():
        tokens[i] = arnum.int2str(tokens[i])
        num_tokens.append(tokens[i])
print("Number to string: ", tokens)
print("Numerical tokens: ", num_tokens)

Number to string:  ['أعلنت', 'المؤسسة', 'العربية', 'لضمان', 'الاستثمار', 'وائتمان', 'الصادرات', 'ضمان', 'حلول', 'المغرب', 'المركز', 'السابع', 'ضمن', 'مؤشر', 'فيتش', 'المركب', 'للمخاطر', 'خلال', 'ألفان و ثلاث و عشرون', 'وأشارت', 'المؤسسة', 'ذاتها', 'بلاغ', 'صادر', 'عنها', 'استقرار', 'التصنيف', 'السيادي', 'لـ4', 'دول', 'عربية', 'تحسن', 'تصنيف', 'الإمارات', 'والسعودية', 'وسلطنة', 'عمان', 'وقطر', 'مقابل', 'تراجع', 'تصنيف', 'المغرب', 'ومصر', 'وتونس', 'ولبنان', 'إضافة', 'تغير', 'الرؤية', 'المستقبلية', 'لـ6', 'دول', 'وفق', 'أهم', 'أربع', 'وكالات', 'عالمية', 'وأفاد', 'البلاغ', 'بأن', 'دول', 'الخليج', 'المغرب', 'فالأردن', 'ومصر', 'استحوذت', 'مقدمة', 'الترتيب', 'عربيا', 'أغلب', 'التقييمات', 'المتعلقة', 'بمؤشرات', 'المخاطر', 'بكل', 'أنواعها', 'أوضحت', 'البيانات', 'المرفقة', 'استقرار', 'تصنيف', 'المغرب', 'سابعا', 'تصنيف', 'الدول', 'العربية', 'مؤشر', 'فيتش', 'المركب', 'لمخاطر', 'الدول', 'بـ55', 'إثنان', 'نقطة', 'المتوسط', 'ثماني و أربعون', 'ثلاث', 'المائة', ').', 'وأكدت', 'ضمان', 'تراجع', 'وضع', 'ا

### Normalization

In [37]:
ArListem = ArabicLightStemmer()
# removing numerical tokens
for i in num_tokens:
    tokens.remove(i)
tokens = [ArListem.normalize(token) for token in tokens]
print("Normalized token list: ", tokens)

Normalized token list:  ['اعلنت', 'المءسسه', 'العربيه', 'لضمان', 'الاستثمار', 'واءتمان', 'الصادرات', 'ضمان', 'حلول', 'المغرب', 'المركز', 'السابع', 'ضمن', 'مءشر', 'فيتش', 'المركب', 'للمخاطر', 'خلال', 'واشارت', 'المءسسه', 'ذاتها', 'بلاغ', 'صادر', 'عنها', 'استقرار', 'التصنيف', 'السيادي', 'ل4', 'دول', 'عربيه', 'تحسن', 'تصنيف', 'الامارات', 'والسعوديه', 'وسلطنه', 'عمان', 'وقطر', 'مقابل', 'تراجع', 'تصنيف', 'المغرب', 'ومصر', 'وتونس', 'ولبنان', 'اضافه', 'تغير', 'الرءيه', 'المستقبليه', 'ل6', 'دول', 'وفق', 'اهم', 'وكالات', 'عالميه', 'وافاد', 'البلاغ', 'بان', 'دول', 'الخليج', 'المغرب', 'فالاردن', 'ومصر', 'استحوذت', 'مقدمه', 'الترتيب', 'عربيا', 'اغلب', 'التقييمات', 'المتعلقه', 'بمءشرات', 'المخاطر', 'بكل', 'انواعها', 'اوضحت', 'البيانات', 'المرفقه', 'استقرار', 'تصنيف', 'المغرب', 'سابعا', 'تصنيف', 'الدول', 'العربيه', 'مءشر', 'فيتش', 'المركب', 'لمخاطر', 'الدول', 'ب55', 'نقطه', 'المتوسط', 'الماءه', ').', 'واكدت', 'ضمان', 'تراجع', 'وضع', 'الدول', 'العربيه', 'مءشرات', 'مخاطر', 'الدوله', 'المتعلقه', 'بعملي

### Stemming

In [38]:
stemmed_tokens = [ArListem.light_stem(token) for token in tokens]
print("Stemmed token list: ", stemmed_tokens)

Stemmed token list:  ['اعلن', 'مءسسه', 'عربيه', 'ضما', 'استثمار', 'ءتما', 'صادر', 'ضم', 'حلول', 'مغرب', 'مركز', 'سابع', 'ضمن', 'مءشر', 'تش', 'مركب', 'مخاطر', 'خلال', 'اشار', 'مءسسه', 'ذا', 'لاغ', 'صادر', 'عن', 'استقرار', 'تصنيف', 'سياد', 'ل4', 'دول', 'عرب', 'حس', 'صنيف', 'امار', 'سعوديه', 'سلط', 'عم', 'قطر', 'مقابل', 'راجع', 'صنيف', 'مغرب', 'مصر', 'ونس', 'بنا', 'ضاف', 'غير', 'رءيه', 'المستقبليه', 'ل6', 'دول', 'فق', 'هم', 'ات', 'عالم', 'فاد', 'بلاغ', 'ان', 'دول', 'خليج', 'مغرب', 'اردن', 'مصر', 'استحوذ', 'مقدم', 'ترتيب', 'عربي', 'غلب', 'تقييم', 'متعلقه', 'مءشر', 'مخاطر', 'كل', 'نواع', 'اوضح', 'بيان', 'مرفقه', 'استقرار', 'صنيف', 'مغرب', 'سابع', 'صنيف', 'دول', 'عربيه', 'مءشر', 'تش', 'مركب', 'مخاطر', 'دول', '55', 'قط', 'متوسط', 'ماءه', ').', 'اكد', 'ضم', 'راجع', 'ضع', 'دول', 'عربيه', 'مءشر', 'مخاطر', 'دوله', 'متعلقه', 'عمل', 'تصدير', 'استثمار', 'مباشره', 'صادره', 'ال', 'ريديندو', 'في', 'مءشر', 'مخاطر', 'تجاره', 'صادر', 'ال', 'يكسي', 'يابانيه', 'شهد', 'متوسط', 'ترتيب', 'عالم', 'هذه', 'دول', 

### Lemmatization

In [39]:
lemmer = lem.Lemmatizer()
lemmatized_tokens = [lemmer.lemmatize(token) for token in tokens]
print("Lemmatized token list: ", lemmatized_tokens)

Lemmatized token list:  ['اعلنت', 'المءسسه', 'العربيه', 'ضمان', 'استثمار', 'واءتمان', 'صادر', 'ضمان', 'حلول', 'مغرب', 'مركز', 'سابع', 'ضمن', 'مءشر', 'فيتش', 'مركب', 'مخاطر', 'خلال', 'واشارت', 'المءسسه', 'ذات', 'بلاغ', 'صادر', 'عن', 'استقرار', 'تصنيف', 'سيادة', 'ل4', 'دول', 'عرب', 'تحسن', 'تصنيف', 'الامارات', 'والسعوديه', 'سلط', 'عمان', 'قطر', 'مقابل', 'تراجع', 'تصنيف', 'مغرب', 'مصر', 'تونس', 'لبن', 'اضافه', 'تغير', 'الرءيه', 'المستقبليه', 'ل6', 'دول', 'وفق', 'اهم', 'كال', 'عالم', 'وافاد', 'بلاغ', 'بان', 'دول', 'خليج', 'مغرب', 'فالاردن', 'مصر', 'استحوذ', 'مقدم', 'ترتيب', 'عرب', 'غلب', 'تقييم', 'المتعلقه', 'بمءشرات', 'مخاطر', 'كل', 'انواعها', 'اوضحت', 'بيان', 'المرفقه', 'استقرار', 'تصنيف', 'مغرب', 'سابع', 'تصنيف', 'دول', 'العربيه', 'مءشر', 'فيتش', 'مركب', 'مخاطر', 'دول', 'ب55', 'نقط', 'متوسط', 'الماءه', ').', 'واكدت', 'ضمان', 'تراجع', 'وضع', 'دول', 'العربيه', 'مءشرات', 'مخاطر', 'الدوله', 'المتعلقه', 'عملة', 'تصدير', 'استثمار', 'المباشره', 'الصادره', 'كال', 'كريديندو', 'في', 'مءشر', 'مخاط

### Stemming vs lemmatization comparison

In [40]:
print("Original token list: ", tokens)
print("Stemmed token list: ", stemmed_tokens)
print("Lemmatized token list: ", lemmatized_tokens)

Original token list:  ['اعلنت', 'المءسسه', 'العربيه', 'لضمان', 'الاستثمار', 'واءتمان', 'الصادرات', 'ضمان', 'حلول', 'المغرب', 'المركز', 'السابع', 'ضمن', 'مءشر', 'فيتش', 'المركب', 'للمخاطر', 'خلال', 'واشارت', 'المءسسه', 'ذاتها', 'بلاغ', 'صادر', 'عنها', 'استقرار', 'التصنيف', 'السيادي', 'ل4', 'دول', 'عربيه', 'تحسن', 'تصنيف', 'الامارات', 'والسعوديه', 'وسلطنه', 'عمان', 'وقطر', 'مقابل', 'تراجع', 'تصنيف', 'المغرب', 'ومصر', 'وتونس', 'ولبنان', 'اضافه', 'تغير', 'الرءيه', 'المستقبليه', 'ل6', 'دول', 'وفق', 'اهم', 'وكالات', 'عالميه', 'وافاد', 'البلاغ', 'بان', 'دول', 'الخليج', 'المغرب', 'فالاردن', 'ومصر', 'استحوذت', 'مقدمه', 'الترتيب', 'عربيا', 'اغلب', 'التقييمات', 'المتعلقه', 'بمءشرات', 'المخاطر', 'بكل', 'انواعها', 'اوضحت', 'البيانات', 'المرفقه', 'استقرار', 'تصنيف', 'المغرب', 'سابعا', 'تصنيف', 'الدول', 'العربيه', 'مءشر', 'فيتش', 'المركب', 'لمخاطر', 'الدول', 'ب55', 'نقطه', 'المتوسط', 'الماءه', ').', 'واكدت', 'ضمان', 'تراجع', 'وضع', 'الدول', 'العربيه', 'مءشرات', 'مخاطر', 'الدوله', 'المتعلقه', 'بعمليات

### Machine learning approach for PoS tagging

In [41]:
pos_tagger = FarasaPOSTagger()
tagged_text =  pos_tagger.tag(text) # [pos_tagger.tag(token) for token in lemmatized_tokens]
print("POS Tagged",tagged_text)

POS Tagged S/S أعلن +ت/V+PRON ال+ مؤسس +ة/DET+NOUN+NSUFF-FS ال+ عربي +ة/DET+ADJ+NSUFF-FS ل+/PREP ضمان/NOUN-MS ال+ استثمار/DET+NOUN-MS و+/CONJ ائتمان/NOUN-MS ال+ صادر +ات/DET+NOUN+NSUFF-FP “/NOUN-MS ضمان/NOUN-MS ”/NOUN-MS عن/PREP حلول/NOUN-FP ال+ مغرب/DET+NOUN-MS في/PREP ال+ مركز/DET+NOUN-MS ال+ سابع/DET+ADJ-MS ضمن/NOUN-MS مؤشر/NOUN-MS “/NOUN-MS فيتش/NOUN-MS ”/NOUN-MS ال+ مركب/DET+NOUN-MS ل+/PREP ال+ مخاطر/DET+NOUN-MP خلال/NOUN-MS 2023/NUM-MP ./PUNC و+/CONJ أشار +ت/V+PRON ال+ مؤسس +ة/DET+NOUN+NSUFF-FS ذات/NOUN-MS +ها/PRON ،/PUNC في/PREP بلاغ/NOUN-MS صادر/ADJ-MS عن/PREP +ها/PRON ،/PUNC إلى/PREP استقرار/NOUN-MS ال+ تصنيف/DET+NOUN-MS ال+ سيادي/DET+ADJ-MS ل/PREP 4/NUM-MP دول/NOUN-FS عربي +ة/ADJ+NSUFF-FS ،/PUNC مع/NOUN-MS تحسن/NOUN-MS تصنيف/NOUN-MS ال+ إمار +ات/DET+NOUN+NSUFF-FP و+/CONJ ال+ سعودي +ة/DET+NOUN+NSUFF-FP و+/CONJ سلطن +ة/NOUN+NSUFF-FS عمان/NOUN-MS و+/CONJ قطر/NOUN-MS ،/PUNC مقابل/NOUN-MS تراجع/NOUN-MS تصنيف/NOUN-MS ال+ مغرب/DET+NOUN-MS و+/CONJ مصر/NOUN-MS و+/CONJ تونس/NOUN-MS و+/

In [42]:
# free up memory
del pos_tagger

### Rule-Based approach for PoS tagging

In [43]:
def tag(token):
    rules_1_2 = [r'^\s*\u0643\u0627\u0644.*\s*$', r'^\s*\u0628\u0627\u0644.*\s*$', r'^\s*\u0641\u0627\u0644.*\s*$', 
                r'^\s*\u0648\u0627\u0644.*\s*$',r'^\s*\w*\u0627\u0626\u0647\b', r'^\s*\w*\u0627\u0626\u0643\b',
                r'^\s*\w*\u0627\u0626\u064a\b', r'^\s*\w*\u0627\u0624\u0643\b', r'^\s*\w*\u0627\u0624\u0647\b',
                r'^\s*\w*\u0627\u0621\u0643\b',r'^\s*\w*\u0627\u0621\u0647\b',r'^\s*\w*\u0647\u0645\u0627\b',
                r'^\s*\w*\u0643\u0645\u0627\b']
    rules_3_4 = [r'^\s*\u0633\u064a.*\s*$', r'^\s*\u0633\u062a.*\s*$', r'^\s*\u0633\u0646.*\s*$', r'^\s*\u0633\u0623.*\s*$',
                r'^\s*\u0633\u0627.*\s*$',r'^\s*\u0644\u0627.*\s*$',r'^\s*\u0644\u0623.*\s*$',r'^\s*\u0644\u0646.*\s*$',
                r'^\s*\u0644\u062a.*\s*$',r'^\s*\u0644\u064a.*\s*$', r'^\s*\w*\u064a\b',r'^\s*\w*\u0647\b',r'^\s*\w*\u0643\b',
                r'^\s*\w*\u0627\b',r'^\s*\w*\u0646\b',r'^\s*\w*\u0648\b']
    rules_5_6 = [r'^\s*\w{3}.*\u0649\s*$', r'^\s*\w{2}\u0648\w{1}\s*$',r'^\s*\w{2}\u0627\u0621\s*$',r'^\s*\w*\u0627\u062a\s*$']
    rule_7 = r'^\s*(\u064a|\u0646)\w*(\u0648\u0646|\u064a\u0646)\s*$'
    rules_8_9 = [r'^\s*[^(\u064a|\u0646)]\w*(\u0648\u0646|\u064a\u0646)\s*$',r'^\s*\u0645\w{1}\u0627\w{2}\s*$',
                 r'^\s*\u0645\w{2}\u064a\w{1}\s*$', r'^\s*\u0645\w{2}\u0627\w{1}\s*$', r'^\s*\u0645\w{1}\u062a\w{2}\s*$',
                 r'^\s*\u0645\u0646\w{3}\s*$',r'^\s*\u0645\w{2}\u0648\w{1}\s*$',r'^\s*\u0645\u062a\w{3}\s*$',
                 r'^\s*\u0645\w{2}(.)\1\s*$']
    rule_10 = [r'^\s*\w{2}\u0627\w{1}\u064a\w{1}\s*$',r'^s*\u0623\w{1}\u0627\w{1}\u064a\w{1}\s*$',
                 r'^\s*\u0645\w{2}\u0627\w{1}\u064a\w{1}\s*$',r'^\s*\w{1}\u0648\u0627\w{1}\u064a\w{1}\s*$']
    rule_11 = [r'^\s*\u0627\u0633\u062a\w{3}\s*$',r'^\s*\u0627\w{1}(.)\u0648\1\w{1}\s*$']
    if any(re.match(pattern, token) for pattern in rules_1_2):
        return 'NOUN'
    elif any(re.match(pattern, token) for pattern in rules_3_4):
        return 'VERB'
    elif any(re.match(pattern, token) for pattern in rules_5_6):
        return 'NOUN'
    elif re.match(rule_7, token):
        return 'VERB'
    elif any(re.match(pattern, token) for pattern in rules_8_9):
        return 'NOUN'
    elif any(re.match(pattern, token) for pattern in rule_10):
        return 'NOUN'
    elif any(re.match(pattern, token) for pattern in rule_11):
        return 'VERB'
    else:
        return 'NOUN'
    
tags = [tag(token) for token in lemmatized_tokens]
print("Lemmatized token list: ", lemmatized_tokens)
print("Custom POS Tagged: ", tags)

Lemmatized token list:  ['اعلنت', 'المءسسه', 'العربيه', 'ضمان', 'استثمار', 'واءتمان', 'صادر', 'ضمان', 'حلول', 'مغرب', 'مركز', 'سابع', 'ضمن', 'مءشر', 'فيتش', 'مركب', 'مخاطر', 'خلال', 'واشارت', 'المءسسه', 'ذات', 'بلاغ', 'صادر', 'عن', 'استقرار', 'تصنيف', 'سيادة', 'ل4', 'دول', 'عرب', 'تحسن', 'تصنيف', 'الامارات', 'والسعوديه', 'سلط', 'عمان', 'قطر', 'مقابل', 'تراجع', 'تصنيف', 'مغرب', 'مصر', 'تونس', 'لبن', 'اضافه', 'تغير', 'الرءيه', 'المستقبليه', 'ل6', 'دول', 'وفق', 'اهم', 'كال', 'عالم', 'وافاد', 'بلاغ', 'بان', 'دول', 'خليج', 'مغرب', 'فالاردن', 'مصر', 'استحوذ', 'مقدم', 'ترتيب', 'عرب', 'غلب', 'تقييم', 'المتعلقه', 'بمءشرات', 'مخاطر', 'كل', 'انواعها', 'اوضحت', 'بيان', 'المرفقه', 'استقرار', 'تصنيف', 'مغرب', 'سابع', 'تصنيف', 'دول', 'العربيه', 'مءشر', 'فيتش', 'مركب', 'مخاطر', 'دول', 'ب55', 'نقط', 'متوسط', 'الماءه', ').', 'واكدت', 'ضمان', 'تراجع', 'وضع', 'دول', 'العربيه', 'مءشرات', 'مخاطر', 'الدوله', 'المتعلقه', 'عملة', 'تصدير', 'استثمار', 'المباشره', 'الصادره', 'كال', 'كريديندو', 'في', 'مءشر', 'مخاط

### Named Entity Recognition

In [44]:
ner = FarasaNamedEntityRecognizer()
ner_tags = ner.recognize(text)
print("NER Tagged text: ",ner_tags)

أعلنت/O المؤسسة/B-ORG العربية/I-ORG لضمان/I-ORG الاستثمار/I-ORG وائتمان/O الصادرات/O “/O ضمان/O ”/O عن/O حلول/O المغرب/B-LOC في/O المركز/O السابع/O ضمن/O مؤشر/O “/O فيتش/B-PERS ”/O المركب/O للمخاطر/O خلال/O 2023/O ./O وأشارت/O المؤسسة/O ذاتها/O ،/O في/O بلاغ/O صادر/O عنها/O ،/O إلى/O استقرار/O التصنيف/O السيادي/O ل/O 4/O دول/O عربية/O ،/O مع/O تحسن/O تصنيف/O الإمارات/B-LOC والسعودية/B-LOC وسلطنة/B-LOC عمان/I-LOC وقطر/B-LOC ،/O مقابل/O تراجع/O تصنيف/O المغرب/B-LOC ومصر/B-LOC وتونس/B-LOC ولبنان/B-LOC ،/O إضافة/O إلى/O تغير/O الرؤية/O المستقبلية/O ل/O 6/O دول/O ،/O وفق/O أهم/O 4/O وكالات/O عالمية/O ./O وأفاد/O البلاغ/O بأن/O دول/O الخليج/B-LOC ثم/O المغرب/B-LOC فالأردن/B-LOC ومصر/B-LOC استحوذت/O على/O مقدمة/O الترتيب/O عربيا/O ،/O في/O أغلب/O التقييمات/O المتعلقة/O بمؤشرات/O المخاطر/O بكل/O أنواعها/O ؛/O فيما/O أوضحت/O البيانات/O المرفقة/O به/O استقرار/O تصنيف/O المغرب/B-LOC سابعا/O في/O تصنيف/O الدول/O العربية/O في/O مؤشر/O “/O فيتش/B-PERS ”/O المركب/O لمخاطر/O الدول/O ب/O 55.2/O نقطة/O 

## References
- Zerrouki, T., (2023). PyArabic: A Python package for Arabic text. Journal of Open Source Software, 8(84), 4886, https://doi.org/10.21105/joss.04886
- Alkhatib, R. M., Zerrouki, T., Shquier, M. M. A., & Balla, A. (2023). Tashaphyne0.4: A new arabic light stemmer based on rhyzome modeling approach. Information Retrieval Journa, 26(14). doi: https://doi.org/10.1007/s10791-023-09429-y
- T. Zerrouki, Qalsadi, Arabic mophological analyzer Library for python.,  https://pypi.python.org/pypi/qalsadi/
- MagedSaeed. (n.d.). GitHub - MagedSaeed/farasapy: A Python implementation of Farasa toolkit. GitHub. https://github.com/MagedSaeed/farasapy?tab=readme-ov-file#want-to-cite
- Hegazi, M. O., Al-Dossari, Y., Al-Yahy, A., Al-Sumari, A., & Hilal, A. (2021). Preprocessing Arabic text on social media. Heliyon, 7(2), e06191. https://doi.org/10.1016/j.heliyon.2021.e06191 
- Sawalha, M., Atwell, E., & Abushariah, M. A. M. (2013). SALMA: Standard Arabic Language Morphological Analysis. 2013 1st International Conference on Communications, Signal Processing, and Their Applications (ICCSPA). https://doi.org/10.1109/iccspa.2013.6487311
- Hjouj, M., Alarabeyyat, A., & Olab, I. (2016). Rule based approach for Arabic part of speech tagging and name entity recognition. International Journal of Advanced Computer Science and Applications, 7(6). https://doi.org/10.14569/ijacsa.2016.070642